<a href="https://colab.research.google.com/github/fazelelham32/AI-Projects/blob/Article-codes/code_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
np.random.seed(42)  # Reproducible

n_trials = 10000

In [ ]:
# Position error: target ~22% reduction
mean_baseline = 0.50
mean_hinc = mean_baseline * 0.78  # 22% reduction
baseline_errors = np.random.normal(mean_baseline, 0.10, n_trials)
hinc_errors = np.random.normal(mean_hinc, 0.08, n_trials)

In [ ]:
# Grasp success: target ~16% increase from ~77.6%
p_baseline = 0.776
p_hinc = p_baseline * 1.16
baseline_success = np.random.binomial(1, p_baseline, n_trials)
hinc_success = np.random.binomial(1, p_hinc, n_trials)

In [ ]:
# Raw data sheet
df_raw = pd.DataFrame({
    'Trial_ID': range(1, n_trials + 1),
    'Baseline_Position_Error': baseline_errors,
    'HINC_Position_Error': hinc_errors,
    'Baseline_Grasp_Success': baseline_success,
    'HINC_Grasp_Success': hinc_success
})

In [ ]:
# Statistical summary
error_reduction = (np.mean(baseline_errors) - np.mean(hinc_errors)) / np.mean(baseline_errors) * 100
success_increase = (np.mean(hinc_success) - np.mean(baseline_success)) / np.mean(baseline_success) * 100

t_error, p_error = stats.ttest_ind(hinc_errors, baseline_errors, equal_var=False)

In [ ]:
# Approximate t-test for proportions
se_prop = np.sqrt(np.mean(hinc_success)*(1-np.mean(hinc_success))/n_trials +
                  np.mean(baseline_success)*(1-np.mean(baseline_success))/n_trials)
t_success = (np.mean(hinc_success) - np.mean(baseline_success)) / se_prop
p_success = stats.norm.sf(abs(t_success)) * 2

df_summary = pd.DataFrame({
    'Metric': [
        'Position Error Reduction (%)',
        'Grasp Success Increase (%)',
        'Baseline Mean Position Error',
        'HINC Mean Position Error',
        'Baseline Grasp Success Rate (%)',
        'HINC Grasp Success Rate (%)',
        't-statistic (Position Error)',
        'p-value (Position Error)',
        't-statistic (Grasp Success approx.)',
        'p-value (Grasp Success approx.)'
    ],
    'Value': [
        round(error_reduction, 2),
        round(success_increase, 2),
        round(np.mean(baseline_errors), 4),
        round(np.mean(hinc_errors), 4),
        round(np.mean(baseline_success)*100, 2),
        round(np.mean(hinc_success)*100, 2),
        round(t_error, 2),
        f"{p_error:.1e}",
        round(t_success, 2),
        f"{p_success:.1e}"
    ]
})
# Save to Excel with two sheets
with pd.ExcelWriter('hinc_simulated_data.xlsx') as writer:
    df_raw.to_excel(writer, sheet_name='Raw_Data', index=False)
    df_summary.to_excel(writer, sheet_name='Statistical_Summary', index=False)

print("File 'hinc_simulated_data.xlsx' generated successfully!")